# BAB 9: Natural Language Processing dengan TensorFlow - Sentiment Analysis

### Pendahuluan

Bab ini membahas tentang **Natural Language Processing (NLP)** dengan fokus pada **Sentiment Analysis** (analisis sentimen). Berbeda dengan bab sebelumnya yang berfokus pada visi komputer (klasifikasi gambar dan segmentasi), bab ini mengeksplorasi cara memproses dan menganalisis data tekstual menggunakan deep learning.

### Konsep NLP Dasar

#### Tugas-Tugas NLP
1. **Stop Word Removal** - Menghilangkan kata-kata yang tidak informatif seperti "dan", "itu", "yang"
2. **Lemmatization** - Mengubah kata ke bentuk dasarnya (contoh: "berjalan" → "jalan")
3. **Part of Speech (PoS) Tagging** - Menandai jenis kata (noun, verb, adjective, dll)
4. **Named Entity Recognition (NER)** - Mengekstrak entitas seperti nama orang, lokasi
5. **Language Modeling** - Memprediksi kata berikutnya dari kata-kata sebelumnya
6. **Sentiment Analysis** - Mengidentifikasi sentimen positif/negatif dari teks
7. **Machine Translation** - Menerjemahkan dari satu bahasa ke bahasa lain

### Dataset: Amazon Video Game Reviews

- Dataset berisi review video game dari Amazon dengan rating 1-5 bintang
- Setiap review memiliki teks review dan rating bintang
- Total ~332,504 verified reviews (dari 497,419 total reviews)
- Dataset sangat **imbalanced**: 83% positif, 17% negatif
- Setelah mapping: 4-5 bintang = positif (label=1), 1-3 bintang = negatif (label=0)

## Tahap 1: Preprocessing Text

### Langkah-Langkah Preprocessing

Proses cleaning teks meliputi:
1. **Lowercase** - Mengubah semua karakter menjadi huruf kecil
2. **Expand Contractions** - Mengubah "isn't" menjadi "is not"
3. **Remove Digits** - Menghilangkan angka-angka
4. **Tokenization** - Memecah teks menjadi kata-kata individual
5. **Remove Stop Words** - Menghilangkan kata-kata umum (TAPI: simpan "not" dan "no")
6. **Lemmatization** - Mengubah kata ke bentuk dasar dengan mempertimbangkan Part of Speech

---

## Program-Program Penting dengan Penjelasan Detail

### 1. Download Dataset

**Penjelasan Ringkas:**
Program ini mengunduh dataset review video game dari Amazon (format JSON.gzip) dari server UCSD dan mengekstrak file terkompresi menjadi JSON untuk diproses. Ini adalah langkah pertama dalam pipelined data yang menggabungkan fetching, caching (jika sudah ada, tidak download lagi), dan dekompresi.

**Langkah-Langkah:**
1. Cek apakah file .gz sudah ada di folder 'data' (cache check)
2. Jika belum, unduh dari URL UCSD
3. Simpan file terkompresi
4. Dekompresi file .gz menjadi JSON
5. Siap untuk loading ke memory

```python
import os
import requests
import gzip
import shutil

# ===== STEP 1: Download file terkompresi =====
# Fungsi os.path.exists() mengecek apakah file sudah ada di local disk
# Ini penting untuk menghindari re-download data yang sama
if not os.path.exists(os.path.join('data','Video_Games_5.json.gz')):  
    # URL dataset dari UCSD (public dataset)
    # Format: .gz (gzip compressed) untuk menghemat bandwidth
    url = "http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/Video_Games_5.json.gz"
    
    # requests.get(url) melakukan HTTP GET request
    # Mengembalikan response object dengan konten file
    r = requests.get(url)
    
    # Buat direktori 'data' jika belum ada
    # os.mkdir() hanya bisa membuat 1 level direktori
    if not os.path.exists('data'):
        os.mkdir('data')
    
    # Tulis (write) konten file binary ke disk
    # 'wb' mode = write binary (karena .gz adalah binary format)
    # r.content = raw bytes dari response
    with open(os.path.join('data','Video_Games_5.json.gz'), 'wb') as f:
        f.write(r.content)
    
    print("✓ File downloaded successfully")

# ===== STEP 2: Extract file terkompresi =====
# File .gz adalah gzip compressed format
# Perlu di-dekompresi sebelum bisa di-baca sebagai JSON
if not os.path.exists(os.path.join('data', 'Video_Games_5.json')):     
    # gzip.open() membuka file terkompresi
    # 'rb' mode = read binary (membaca bytes terkompresi)
    with gzip.open(os.path.join('data','Video_Games_5.json.gz'), 'rb') as f_in:
        # Buka file output untuk menulis data yang sudah didekompresi
        # 'wb' mode = write binary
        with open(os.path.join('data','Video_Games_5.json'), 'wb') as f_out:
            # shutil.copyfileobj() copy file secara efficient (chunk by chunk)
            # Lebih baik daripada read semua ke memory lalu write
            # Hemat memory untuk file besar
            shutil.copyfileobj(f_in, f_out)
    
    print("✓ File extracted successfully")
else:
    print("✓ File already exists, skipping download")
```

---

### 2. Load dan Explore Data

**Penjelasan Ringkas:**
Program ini membaca file JSON yang sudah diekstrak ke pandas DataFrame, melakukan data exploration (melihat distribusi rating, jumlah review verified vs unverified), dan membuat label biner untuk sentiment classification (4-5 stars = positif, 1-3 stars = negatif).

**Langkah-Langkah:**
1. Load JSON file dengan pandas
2. Pilih kolom yang relevan
3. Remove missing/empty reviews
4. Filter hanya verified reviews (biar lebih reliable)
5. Periksa class distribution (data imbalance)
6. Convert rating 5-star menjadi binary labels
7. Shuffle data untuk training yang lebih baik

```python
import pandas as pd

# ===== STEP 1: Load JSON file ke DataFrame =====
# pd.read_json() membaca JSON file
# lines=True: setiap baris adalah satu JSON object (JSONL format)
# orient='records': mengkonversi JSON objects menjadi row records
review_df = pd.read_json(
    os.path.join('data', 'Video_Games_5.json'),
    lines=True,  # JSONL format (JSON Lines)
    orient='records'
)

# ===== STEP 2: Pilih kolom yang relevan =====
# Dataset punya banyak kolom (reviewerID, asin, verified, overall, reviewText, dll)
# Kita hanya butuh: rating (overall), verified status, dan review text
review_df = review_df[["overall", "verified", "reviewTime", "reviewText"]]

print("Data shape:", review_df.shape)  # Berapa banyak rows dan columns
print(review_df.head())  # Tampilkan 5 rows pertama

# ===== STEP 3: Remove empty atau null reviews =====
# Beberapa review mungkin kosong atau cuma whitespace
# isna() return True jika value adalah null/NaN
# ~(tilde) adalah NOT operator
review_df = review_df[~review_df["reviewText"].isna()]  # Remove null

# .str accessor mengakses string methods
# .strip() remove leading/trailing whitespace
# .str.len() > 0 filter hanya reviews dengan content
review_df = review_df[review_df["reviewText"].str.strip().str.len()>0]

print("After removing empty:", review_df.shape)

# ===== STEP 4: Filter verified reviews only =====
# Verified reviews = dari pembeli asli yang terbukti beli produk
# Lebih reliable daripada review dari sembarang orang
# .loc[condition, :] = filter rows based on condition
verified_df = review_df.loc[review_df["verified"], :]

print("Verified reviews count:", len(verified_df))

# ===== STEP 5: Check class distribution =====
# Lihat berapa banyak review untuk setiap rating
# PENTING: untuk mendeteksi class imbalance problem
print("\nRating distribution:")
print(verified_df["overall"].value_counts())

# Output menunjukkan:
# 5 stars: 222,335 reviews (paling banyak)
# 4 stars: 54,878 reviews
# 3 stars: 27,973 reviews
# 1 star:  15,200 reviews
# 2 stars: 12,118 reviews
# IMBALANCED! 5-star jauh lebih banyak dari yang lain

# ===== STEP 6: Create binary labels =====
# Sentiment classification = 2 classes
# Assumption: 4-5 stars = POSITIF (label=1), 1-3 stars = NEGATIF (label=0)
# .map() fungsi pandas untuk transform values berdasarkan dictionary
verified_df["label"] = verified_df["overall"].map({
    5: 1,  # 5 stars -> positive
    4: 1,  # 4 stars -> positive
    3: 0,  # 3 stars -> negative
    2: 0,  # 2 stars -> negative
    1: 0   # 1 star -> negative
})

print("\nLabel distribution after mapping:")
print(verified_df["label"].value_counts())
# Output: 277,213 positive (label=1), 55,291 negative (label=0)
# Still imbalanced! 83% positive, 17% negative

# ===== STEP 7: Shuffle data =====
# PENTING: untuk menghindari bias karena data mungkin sorted by rating
# .sample(frac=1.0) = sample 100% dari data (shuffle semua)
# random_state=42 = fixed seed untuk reproducibility
verified_df = verified_df.sample(frac=1.0, random_state=42)

print("\n✓ Data shuffled")

# ===== STEP 8: Separate inputs dan labels =====
# Siapkan untuk preprocessing step berikutnya
inputs, labels = verified_df["reviewText"], verified_df["label"]

print(f"\nTotal samples: {len(inputs)}")
print(f"Positive: {(labels==1).sum()}, Negative: {(labels==0).sum()}")
```

---

### 3. Download NLTK Resources

**Penjelasan Ringkas:**
NLTK (Natural Language Toolkit) memerlukan beberapa resource tambahan untuk preprocessing text seperti:
- **stopwords**: daftar kata-kata umum yang tidak informatif
- **wordnet**: database untuk lemmatization (mencari base form kata)
- **punkt**: tokenizer untuk memecah text menjadi kata-kata
- **pos_tag**: untuk mengenali jenis kata (noun, verb, dll)

Program ini download semua resources yang diperlukan.

```python
import nltk

# ===== Download NLTK resources =====
# NLTK library comes with many tools, tapi resources (data) perlu di-download terpisah

# averaged_perceptron_tagger: model untuk Part-of-Speech tagging
# Digunakan untuk mengidentifikasi apakah kata adalah noun, verb, adjective, dll
nltk.download('averaged_perceptron_tagger', download_dir='nltk')

# wordnet: lexical database untuk lemmatization
# Contain base forms dari kata-kata (contoh: "running" -> "run")
nltk.download('wordnet', download_dir='nltk')

# omw-1.4: Open Multilingual Wordnet untuk berbagai bahasa
nltk.download('omw-1.4', download_dir='nltk')

# stopwords: daftar kata-kata umum per bahasa
# Untuk English, termasuk: "the", "a", "and", "is", "it", dll
nltk.download('stopwords', download_dir='nltk')

# punkt: sentence dan word tokenizer
# Digunakan untuk memecah teks menjadi sentences/words
nltk.download('punkt', download_dir='nltk')

# Tambahkan path local ke NLTK data path
# Agar NLTK mencari resources di folder lokal juga
nltk.data.path.append(os.path.abspath('nltk'))

print("✓ All NLTK resources downloaded")
```

---

### 4. Text Cleaning Function

**Penjelasan Ringkas:**
Ini adalah **core function** untuk membersihkan text sebelum masuk ke model. Function melakukan:
1. Lowercase semua kata
2. Expand contractions (aren't → are not)
3. Hapus digits
4. Tokenization (split ke words)
5. Hapus stop words (tapi PERTAHANKAN "not"/"no" karena penting untuk sentiment!)
6. Lemmatization (convert ke base form dengan mempertimbangkan part-of-speech)

Output: list of clean words yang siap untuk embedding/encoding.

```python
import re
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import string

# ===== Setup preprocessing components =====
# Get English stopwords tapi EXCLUDE 'not' dan 'no' (penting untuk sentiment!)
# "This is NOT a great game" vs "This is a great game" = completely different meaning
EN_STOPWORDS = set(stopwords.words('english')) - {'not', 'no'}

# WordNetLemmatizer: mengubah kata ke base form
# Contoh: "running" -> "run", "better" -> "good", "players" -> "player"
lemmatizer = WordNetLemmatizer()

def clean_text(doc):
    """
    Clean dan preprocess satu review text
    Input: doc = raw text string (review)
    Output: list of cleaned words
    
    Process:
    1. Lowercase
    2. Expand contractions (isn't -> is not)
    3. Remove shortened forms ('ll, 're, 'd, 've)
    4. Remove digits (1, 2, 3, dll)
    5. Tokenize dan remove stop words
    6. Lemmatize (convert to base form)
    """
    
    # ===== STEP 1: Lowercase =====
    # Mengubah semua character menjadi huruf kecil
    # Agar "Game" dan "game" diperlakukan sebagai kata yang sama
    doc = doc.lower()
    
    # ===== STEP 2: Expand contractions =====
    # "can't" -> "can not", "don't" -> "do not", dll
    # Penting untuk mempertahankan semantic meaning
    # Menggunakan string.replace() untuk simple replacement
    doc = doc.replace("n't ", ' not ')
    
    # ===== STEP 3: Remove shortened forms =====
    # "won't" -> "wo not" (akan dihapus "wo" di step stop word removal)
    # "I'll" -> "I", "they're" -> "they", dll
    # Menggunakan regex (regular expression) untuk pattern matching
    # Pattern: \\'ll (escaped quote) atau \\'re atau \\'d atau \\'ve
    doc = re.sub(r"(?:\\'ll |\\'re |\\'d |\\'ve )", " ", doc)
    
    # ===== STEP 4: Remove digits =====
    # Hapus semua angka (0-9)
    # Regex pattern: \d+ (satu atau lebih digit)
    # Replace dengan empty string ""
    doc = re.sub(r"\d+", "", doc)
    
    # ===== STEP 5: Tokenization dan remove stop words =====
    # word_tokenize(doc) memecah text menjadi list of words
    # List comprehension: filter hanya words yang BUKAN stop words dan BUKAN punctuation
    # string.punctuation = "!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~"
    tokens = [
        w for w in word_tokenize(doc)
        if w not in EN_STOPWORDS and w not in string.punctuation
    ]
    
    # ===== STEP 6: Lemmatization =====
    # Pertama, get Part-of-Speech tag untuk setiap word
    # nltk.pos_tag() return list of (word, POS_tag) tuples
    # Contoh: [('running', 'VBG'), ('faster', 'RBR'), ('than', 'IN')]
    pos_tags = pos_tag(tokens)
    
    # Lemmatize hanya NOUNS dan VERBS (karena paling berubah-ubah)
    # Adjectives, adverbs, dll di-skip untuk menghemat computation
    clean_text_result = [
        # lemmatizer.lemmatize(word, pos=POS_tag)
        # pos parameter penting! lemmatization logic berbeda untuk noun vs verb
        lemmatizer.lemmatize(w, pos=p[0].lower())
        if p[0]=='N' or p[0]=='V'  # N = Noun, V = Verb (POS tag dari Penn Treebank)
        else w  # Jika bukan noun/verb, keep original word
        for (w, p) in pos_tags
    ]
    
    return clean_text_result

# ===== Example usage =====
sample_doc = "She sells seashells by the seashore. I can't believe it's not butter!"
print("Before cleaning:", sample_doc)
print("After cleaning:", clean_text(sample_doc))
# Output: ['sell', 'seashell', 'seashore', 'not', 'believe', 'butter']

# ===== STEP 7: Apply to entire dataset =====
# Gunakan pandas .apply() untuk apply function ke setiap row
# lambda x: clean_text(x) = anonymous function yang call clean_text untuk setiap review
# ⚠️ WARNING: Ini sangat lambat! Bisa ambil 30-60 menit tergantung CPU
# Improvement: gunakan parallel processing dengan joblib atau multiprocessing
print("\nCleaning all reviews (ini akan lambat, tunggu ~1 jam)...")
inputs = inputs.apply(lambda x: clean_text(x))

# ===== STEP 8: Save untuk future use =====
# Menyimpan ke pickle file untuk menghindari re-processing
# Pickle = Python's native serialization format
inputs.to_pickle(os.path.join('data','sentiment_inputs.pkl'))
labels.to_pickle(os.path.join('data','sentiment_labels.pkl'))

print("✓ Data cleaned dan saved")
```

---

### 5. Train/Validation/Test Split Function

**Penjelasan Ringkas:**
Ini adalah **CRITICAL** function untuk proper data handling. Karena dataset imbalanced (83% positif, 17% negatif), kita perlu:
1. Split train/val/test SEBELUM melakukan analysis (vocab, sequence length, dll)
2. Buat validation dan test set BALANCED (50% pos, 50% neg)
3. Sisa data untuk training (masih imbalanced, tapi itu OK)
4. Ini mencegah **DATA LEAKAGE** - masalah di mana validation/test data bocor ke training

**Mengapa penting:** Jika kita compute vocab size dari semua data (termasuk val/test), maka model akan "tahu" tentang val/test data, dan performance metrics jadi tidak akurat.

```python
def train_valid_test_split(inputs, labels, train_fraction=0.8, random_seed=42):
    """
    Split data menjadi train/validation/test dengan strategi balanced
    
    PENTING: Data dengan 83% positif, 17% negatif (IMBALANCED)
    Strategi:
    1. Pisahkan positive dan negative indices
    2. Sample balanced val dan test set dari masing-masing class
    3. Sisa untuk training (akan lebih imbalanced, tapi OK)
    4. Ini ensure val/test set balanced untuk fair evaluation
    
    Args:
        inputs: Series of cleaned texts (list of words per review)
        labels: Series of binary labels (0 or 1)
        train_fraction: berapa % untuk training (default 0.8 = 80%)
        random_seed: seed untuk reproducibility
    
    Returns:
        Tuple of ((tr_x, tr_y), (v_x, v_y), (ts_x, ts_y))
    """
    
    # ===== STEP 1: Separate positive dan negative indices =====
    # Perlu handle positif dan negatif secara terpisah karena imbalanced
    # labels.loc[(labels==0)].index = indices dari semua negative examples
    neg_indices = pd.Series(labels.loc[(labels==0)].index)  
    pos_indices = pd.Series(labels.loc[(labels==1)].index)  
    
    print(f"Total negative: {len(neg_indices)}, Total positive: {len(pos_indices)}")
    
    # ===== STEP 2: Calculate validation dan test size =====
    # Formula: n_valid = min(neg_count, pos_count) × ((1-train_frac)/2)
    # Menggunakan min(neg, pos) agar seimbang dari class yang lebih sedikit
    # ((1-train_frac)/2) karena 1-train_frac untuk val+test, dibagi 2 untuk val dan test
    # Contoh: train=0.8 -> (1-0.8)/2 = 0.1 = 10% untuk val, 10% untuk test
    n_valid = int(min([len(neg_indices), len(pos_indices)])
       * ((1-train_fraction)/2.0))
    n_test = n_valid  # Validation dan test size sama
    
    print(f"Validation size: {n_valid}, Test size: {n_test}")
    
    # ===== STEP 3: Sample negative indices =====
    # Random sample untuk test set
    neg_test_inds = neg_indices.sample(n=n_test, random_state=random_seed)
    
    # Random sample untuk validation set dari negative yang belum diambil
    # ~neg_indices.isin(neg_test_inds) = filter indices yang TIDAK di test set
    neg_valid_inds = neg_indices.loc[~neg_indices.isin(
        neg_test_inds)].sample(n=n_test, random_state=random_seed)
    
    # Sisa negative indices untuk training
    neg_train_inds = neg_indices.loc[~neg_indices.isin(
        neg_test_inds.tolist()+neg_valid_inds.tolist())]
    
    # ===== STEP 4: Sample positive indices (sama seperti negative) =====
    pos_test_inds = pos_indices.sample(n=n_test, random_state=random_seed)
    pos_valid_inds = pos_indices.loc[~pos_indices.isin(
        pos_test_inds)].sample(n=n_test, random_state=random_seed)
    pos_train_inds = pos_indices.loc[~pos_indices.isin(
        pos_test_inds.tolist()+pos_valid_inds.tolist())]
    
    # ===== STEP 5: Create actual datasets =====
    # Combine negative dan positive indices untuk setiap set
    # .sample(frac=1.0) = shuffle data sebelum return
    
    tr_x = inputs.loc[neg_train_inds.tolist() + pos_train_inds.tolist()].sample(
        frac=1.0, random_state=random_seed)
    tr_y = labels.loc[neg_train_inds.tolist() + pos_train_inds.tolist()].sample(
        frac=1.0, random_state=random_seed)
    
    v_x = inputs.loc[neg_valid_inds.tolist() + pos_valid_inds.tolist()].sample(
        frac=1.0, random_state=random_seed)
    v_y = labels.loc[neg_valid_inds.tolist() + pos_valid_inds.tolist()].sample(
        frac=1.0, random_state=random_seed)
    
    ts_x = inputs.loc[neg_test_inds.tolist() + pos_test_inds.tolist()].sample(
        frac=1.0, random_state=random_seed)
    ts_y = labels.loc[neg_test_inds.tolist() + pos_test_inds.tolist()].sample(
        frac=1.0, random_state=random_seed)
    
    # ===== STEP 6: Verify splits =====
    print(f'\n✓ Training data: {len(tr_x)} samples')
    print(f'  Positive: {(tr_y==1).sum()}, Negative: {(tr_y==0).sum()}')
    print(f'✓ Validation data: {len(v_x)} samples')
    print(f'  Positive: {(v_y==1).sum()}, Negative: {(v_y==0).sum()}')
    print(f'✓ Test data: {len(ts_x)} samples')
    print(f'  Positive: {(ts_y==1).sum()}, Negative: {(ts_y==0).sum()}')
    
    return (tr_x, tr_y), (v_x, v_y), (ts_x, ts_y)

# ===== Usage =====
(tr_x, tr_y), (v_x, v_y), (ts_x, ts_y) = train_valid_test_split(
    inputs, labels, train_fraction=0.8, random_seed=42
)
```

---

### 6. Analyze Vocabulary

**Penjelasan Ringkas:**
Program ini menganalisis vocabulary dari **training set SAJA** (tidak val/test set!). Tujuannya:
1. Lihat berapa banyak unique words
2. Lihat distribusi frequency (apakah ada words yang jarang muncul?)
3. Tentukan **vocabulary size hyperparameter**: berapa banyak top words yang akan digunakan di model

Insight: Menyimpan semua 133K unique words terlalu boros. Cukup ambil top 11,800 words yang muncul ≥25 kali. Words lain di-treat sebagai "unknown" token.

```python
from collections import Counter

# ===== STEP 1: Create word frequency dictionary =====
# tr_x adalah list of list of words (setelah cleaning)
# [['great', 'game'], ['love', 'it'], ['amazing', 'product'], ...]
#
# Flatten semua words dari semua reviews menjadi satu list
# List comprehension: iterate setiap doc, iterate setiap word dalam doc
data_list = [w for doc in tr_x for w in doc]

# Counter = special dict yang count frequency
# Hasilnya: {'game': 5234, 'great': 3421, 'love': 2987, ...}
cnt = Counter(data_list)

print(f"Total unique words: {len(cnt)}")
print(f"Total word occurrences: {len(data_list)}")

# ===== STEP 2: Create frequency dataframe =====
# Ubah counter menjadi pandas Series untuk easier manipulation
freq_df = pd.Series(
    list(cnt.values()),
    index=list(cnt.keys())
).sort_values(ascending=False)  # Sort by frequency descending

# ===== STEP 3: Inspect top words =====
print("\nTop 20 most frequent words:")
print(freq_df.head(n=20))
# Expected: game, great, love, fun, play, etc.

# ===== STEP 4: Get summary statistics =====
print("\nFrequency statistics:")
print(freq_df.describe())
# Output:
# count  133714 (total unique words)
# mean   ~76 (rata-rata frequency)
# std    ~1754 (standard deviation)
# min    1 (some words appear only once)
# max    5234 (most common word appears 5234 times)

# ===== STEP 5: Determine vocabulary size =====
# DECISION: Gunakan hanya words yang muncul ≥25 kali
# Alasan:
# - Words yang rare (muncul 1-2 kali) tidak informatif, cuma menambah noise
# - Mengurangi vocabulary size -> lebih kecil embedding matrix -> faster training
# - Hasil: ~11,800 words dipilih dari 133,714 unique words
n_vocab = (freq_df >= 25).sum()

print(f"\n✓ Vocabulary size (freq >= 25): {n_vocab}")
print(f"✓ Percentage of all words: {n_vocab/len(freq_df)*100:.2f}%")

# ===== STEP 6: Visualize distribution =====
import matplotlib.pyplot as plt

# Plot histogram dari word frequencies
plt.figure(figsize=(12, 5))

# Subplot 1: Frequency distribution (semua words)
plt.subplot(1, 2, 1)
plt.hist(freq_df.values, bins=100, edgecolor='black')
plt.xlabel('Word Frequency')
plt.ylabel('Count')
plt.title('Word Frequency Distribution (All Words)')
plt.xscale('log')  # Log scale karena distribution sangat skewed

# Subplot 2: Vocabulary size vs frequency threshold
plt.subplot(1, 2, 2)
freq_thresholds = range(1, 100)
vocab_sizes = [(freq_df >= t).sum() for t in freq_thresholds]
plt.plot(freq_thresholds, vocab_sizes)
plt.xlabel('Minimum Frequency Threshold')
plt.ylabel('Vocabulary Size')
plt.title('Vocabulary Size vs Frequency Threshold')
plt.axvline(x=25, color='red', linestyle='--', label='Chosen threshold (25)')
plt.legend()

plt.tight_layout()
plt.show()
```

---

### 7. Analyze Sequence Length

**Penjelasan Ringkas:**
Program ini menganalisis **panjang setiap review** (jumlah words setelah cleaning). Ini penting karena:
1. LSTM bisa handle variable-length sequences, TAPI tidak efisien jika panjang sangat berbeda-beda
2. Dengan bucketing (grouping reviews dengan panjang similar), kita bisa reduce padding waste dan improve efficiency
3. Dari analysis, kita tentukan bucket boundaries untuk grouping reviews

Insight: Kebanyakan reviews 5-16 kata. Ada beberapa outliers dengan 50+ kata. Gunakan bucket boundaries [5, 15] untuk membagi menjadi 3 groups.

```python
# ===== STEP 1: Get sequence length untuk setiap review =====
# tr_x adalah pandas Series of lists
# .str.len() tidak bisa langsung untuk list, jadi kita harus extract length dulu
seq_length_ser = tr_x.apply(lambda x: len(x))  # length of each review

print(f"Sample sequence lengths: {seq_length_ser.head(10).tolist()}")

# ===== STEP 2: Get percentiles =====
# Percentile = nilai di mana X% dari data lebih kecil
# 10th percentile = 10% data lebih kecil, 90% lebih besar
# 90th percentile = 90% data lebih kecil, 10% lebih besar
p_10 = seq_length_ser.quantile(0.1)   # 10th percentile
p_25 = seq_length_ser.quantile(0.25)  # 25th percentile
p_50 = seq_length_ser.quantile(0.50)  # median (50th percentile)
p_75 = seq_length_ser.quantile(0.75)  # 75th percentile
p_90 = seq_length_ser.quantile(0.90)  # 90th percentile

print(f"\nSequence length percentiles:")
print(f"  10th: {p_10:.0f} words")
print(f"  25th: {p_25:.0f} words")
print(f"  50th (median): {p_50:.0f} words")
print(f"  75th: {p_75:.0f} words")
print(f"  90th: {p_90:.0f} words")

# Expected output (dari Bab):
# 10th: 1, 25th: 3, 50th: 10, 75th: 16, 90th: 74

# ===== STEP 3: Get summary statistics (excluding outliers) =====
# Untuk better summary, hapus extreme outliers
# Gunakan 10th-90th percentile range
filtered_lengths = seq_length_ser[
    (seq_length_ser >= p_10) & (seq_length_ser < p_90)
]

print(f"\nDetailed statistics (10th-90th percentile):")
print(filtered_lengths.describe(percentiles=[0.33, 0.66]))

# Output akan show:
# 33% of reviews <= 5 words (SHORT)
# 66% of reviews <= 16 words (MEDIUM)
# 90% of reviews <= 74 words (LONG)

# ===== STEP 4: Determine bucketing strategy =====
# Bucketing = group reviews dengan similar length untuk efficient batching
# Bucket boundaries = split points

# Strategy: Gunakan percentiles sebagai bucket boundaries
# bucket_boundaries = [p_33, p_66] = [5, 16]
# Artinya:
# Bucket 1: 0-5 words (short reviews) - batch_size=128
# Bucket 2: 5-15 words (medium reviews) - batch_size=128  
# Bucket 3: 15+ words (long reviews) - batch_size=128

bucket_boundaries = [5, 15]

print(f"\n✓ Bucket boundaries selected: {bucket_boundaries}")
print("  - Bucket 1 (0-5 words): batch_size=128")
print("  - Bucket 2 (5-15 words): batch_size=128")
print("  - Bucket 3 (15+ words): batch_size=128")

# ===== STEP 5: Visualize sequence length distribution =====
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

# Subplot 1: Histogram
plt.subplot(1, 2, 1)
plt.hist(seq_length_ser, bins=100, edgecolor='black')
plt.xlabel('Sequence Length (number of words)')
plt.ylabel('Frequency')
plt.title('Distribution of Review Lengths')
plt.axvline(x=p_50, color='red', linestyle='--', label=f'Median: {p_50:.0f}')
plt.legend()

# Subplot 2: CDF (Cumulative Distribution Function)
plt.subplot(1, 2, 2)
sorted_lengths = np.sort(seq_length_ser)
cdf = np.arange(1, len(sorted_lengths)+1) / len(sorted_lengths)
plt.plot(sorted_lengths, cdf)
plt.axvline(x=5, color='red', linestyle='--', alpha=0.5, label='Bucket 1-2 boundary')
plt.axvline(x=15, color='orange', linestyle='--', alpha=0.5, label='Bucket 2-3 boundary')
plt.xlabel('Sequence Length')
plt.ylabel('Cumulative Probability')
plt.title('CDF of Review Lengths')
plt.legend()

plt.tight_layout()
plt.show()
```

---

### 8. Keras Tokenizer

**Penjelasan Ringkas:**
Keras Tokenizer mengkonversi **text (words) menjadi numbers (IDs)** karena neural networks hanya bisa memproses numbers, bukan strings. Tokenizer:
1. Fits pada training data untuk build word-to-ID mapping dictionary
2. Menggunakan mapping ini untuk convert semua reviews (train/val/test) menjadi sequences of IDs
3. Unknown words (tidak di vocab) di-assign ID khusus (OOV = Out Of Vocabulary)

Contoh:
```
Text: ['great', 'game', 'love', 'it']
Sequence IDs: [14, 2, 157, 96]
```

```python
from tensorflow.keras.preprocessing.text import Tokenizer

# ===== STEP 1: Create Tokenizer object =====
# Tokenizer adalah dict-like object yang store word-to-ID mapping
tokenizer = Tokenizer(
    num_words=n_vocab,  # Hanya gunakan top 11,800 words (dari vocab analysis)
    oov_token='unk',    # OOV = Out Of Vocabulary, untuk unknown words
    lower=False,        # Already lowercase di preprocessing, jadi set ke False
    # filters = characters yang akan di-remove saat tokenization
    # Kita sudah remove ini di cleaning, tapi keras tokenizer punya default
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    split=' ',          # Split by space (sudah dilakukan di preprocessing)
    char_level=False    # False = word level (bukan character level)
)

# ===== STEP 2: Fit tokenizer on training data =====
# PENTING: Fit hanya pada TRAINING SET!
# Jangan fit pada validation/test set (data leakage)
# .fit_on_texts() melihat semua words di training data dan build vocab dictionary
tokenizer.fit_on_texts(tr_x.tolist())

print("✓ Tokenizer fitted on training data")

# ===== STEP 3: Inspect word-to-ID mapping =====
# tokenizer.word_index = dictionary {word: ID}
# Contoh struktur:
# {'game': 2, 'great': 3, 'love': 4, 'fun': 5, ...}
# Note: ID 1 reserved untuk padding, ID 0 reserved untuk masking

print(f"\nWord-to-ID mapping examples:")
print(f"  'game' -> ID {tokenizer.word_index.get('game', 'unknown')}")
print(f"  'great' -> ID {tokenizer.word_index.get('great', 'unknown')}")
print(f"  'play' -> ID {tokenizer.word_index.get('play', 'unknown')}")

# tokenizer.index_word = reverse mapping {ID: word}
# Untuk debugging/visualization
print(f"\nID-to-word mapping examples:")
print(f"  ID 2 -> '{tokenizer.index_word.get(2, 'unknown')}'")
print(f"  ID 3 -> '{tokenizer.index_word.get(3, 'unknown')}'")

# ===== STEP 4: Convert texts to sequences =====
# texts_to_sequences() menggunakan mapping untuk convert words to IDs
# Input: list of list of words
# Output: list of list of IDs

print("\nConverting texts to sequences...")

tr_x_seq = tokenizer.texts_to_sequences(tr_x.tolist())
v_x_seq = tokenizer.texts_to_sequences(v_x.tolist())
ts_x_seq = tokenizer.texts_to_sequences(ts_x.tolist())

# ===== STEP 5: Inspect converted sequences =====
print(f"\nExample original text: {tr_x.iloc[0]}")
print(f"Example converted sequence: {tr_x_seq[0]}")

# ===== STEP 6: Handle unknown words =====
# OOV token 'unk' akan di-assign ID sebagai:
# oov_token_id = num_words + 1 = 11,801
# Jadi words yang tidak di vocabulary akan punya ID 11,801
# Test: review dengan unknown word
test_review = ['unknown_word', 'great', 'game']
test_seq = tokenizer.texts_to_sequences([test_review])
print(f"\nTest with unknown word:")
print(f"  Text: {test_review}")
print(f"  Sequence: {test_seq}")
# Output: [[11801, <great_id>, <game_id>]]
# (unknown_word -> 11801, great -> standard_id, game -> standard_id)

print("✓ Text-to-sequence conversion complete")
```

---

## Kesimpulan Tahap Preprocessing

Setelah semua program di atas selesai, kita punya:

1. **Clean text data** - dengan lemmatization, stop word removal, dll
2. **Proper train/val/test split** - balanced val/test untuk fair evaluation
3. **Vocabulary** - top 11,800 words yang muncul ≥25 kali
4. **Sequence length analysis** - untuk bucketing strategy
5. **Tokenized sequences** - numbers siap untuk neural network

Next: TensorFlow data pipeline dengan bucketing, LSTM models, dan word embeddings...
